<a href="https://colab.research.google.com/github/raghavkaushik/DeepLearning/blob/main/AutoEncoders_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, GlobalMaxPooling2D, Flatten, Input, Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model

In [4]:
(Xtrain,Ytrain),(Xtest,Ytest)=mnist.load_data()
image_size=Xtrain.shape[1]
Xtrain,Xtest=Xtrain/255.0, Xtest/255.0
Xtrain=np.reshape(Xtrain,[-1,image_size,image_size,1])
Xtest=np.reshape(Xtest,[-1,image_size,image_size,1])

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
input_shape=(image_size,image_size,1)
batch_size=32
kernel_size=3
latent_dim=16

In [12]:
inputs=Input(shape=input_shape, name='encoder_input')
x=Conv2D(32,(3,3),activation='relu',strides=2, padding='same')(inputs)
x=Conv2D(64,(3,3),activation='relu',strides=2, padding='same')(x)

shape=K.int_shape(x)

x=Flatten()(x)
latent=Dense(latent_dim, name='latent_vector')(x)
encoder=Model(inputs,latent,name='encoder')
encoder.summary()
#plot_model(encoder,to_file='encoder.png', show_shapes=True)

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 14, 14, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 flatten_3 (Flatten)         (None, 3136)              0         
                                                                 
 latent_vector (Dense)       (None, 16)                50192     
                                                                 
Total params: 69,008
Trainable params: 69,008
Non-trainable params: 0
_________________________________________________________________


In [13]:
latent_inputs=Input(shape=(latent_dim,), name='decoder_input')
x=Dense(shape[1]*shape[2]*shape[3])(latent_inputs)
x=Reshape((shape[1],shape[2],shape[3]))(x)

x=Conv2DTranspose(64,(3,3),activation='relu',strides=2,padding='same')(x)
x=Conv2DTranspose(32,(3,3),activation='relu',strides=2,padding='same')(x)
outputs=Conv2DTranspose(1, (3,3), activation='sigmoid', padding='same', name='decoder_output')(x)

decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 3136)              53312     
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 decoder_output (Conv2DTrans  (None, 28, 28, 1)        289 

In [14]:
autoencoder=Model(inputs,decoder(encoder(inputs)), name='autoencoder')
autoencoder.summary()
autoencoder.compile(loss='mse', optimizer='adam')

autoencoder.fit(Xtrain,
                Xtrain,
                validation_data=(Xtest, Xtest),
                epochs=1,
                batch_size=batch_size)


Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder (Functional)        (None, 16)                69008     
                                                                 
 decoder (Functional)        (None, 28, 28, 1)         108993    
                                                                 
Total params: 178,001
Trainable params: 178,001
Non-trainable params: 0
_________________________________________________________________
1875/1875 [==============================] - 112s 59ms/step - loss: 0.0212 - val_loss: 0.0105


In [ ]:
x_decoded = autoencoder.predict(Xtest)
imgs = np.concatenate([Xtest[:8], x_decoded[:8]])
imgs = imgs.reshape((4, 4, image_size, image_size))
imgs = np.vstack([np.hstack(i) for i in imgs])
plt.figure()
plt.axis('off')
plt.title('Input: 1st 2 rows, Decoded: last 2 rows')
plt.imshow(imgs, interpolation='none', cmap='gray')
plt.savefig('input_and_decoded.png')
plt.show()